In [ ]:
import sys
sys.path.append('../../../Fedot.Industrial')

import os
import numpy as np
from torch.utils.data import Subset
from torchvision.transforms import Compose, Resize, Normalize, ToTensor
from core.architecture.datasets.splitters import get_dataset_mean_std, split_data, undersampling, dataset_info

DATASETS_ROOT = '/media/n31v/data/datasets'

## ImageNet
mean=(0.13066,), std=(0.3081,)

In [ ]:
from torchvision.datasets import ImageNet

imagenet_ds = ImageNet('/media/n31v/data/datasets/imagenet-object-localization-challenge')

## MNIST
mean=(0.13066,), std=(0.3081,)

In [ ]:
from torchvision.datasets import MNIST

mnist_ds = MNIST(
    root=DATASETS_ROOT,
    transform=ToTensor(),
)
mean, std = get_dataset_mean_std(mnist_ds)
dataset_info(mnist_ds)
print(f'{mean=}, {std=}')

In [ ]:
mnist_ds = undersampling(mnist_ds, n=5000)

folds = []
for i in range(5):
    f1, f2 = split_data(mnist_ds, 2)
    folds.append(np.array([mnist_ds.indices[f1], mnist_ds.indices[f2]]))
folds = np.array(folds)
np.save(os.path.join(DATASETS_ROOT, 'MNIST', 'folds'), folds)
print(folds.shape)

In [ ]:
folds = np.load(os.path.join(DATASETS_ROOT, 'MNIST', 'folds.npy'))

mnist_ds = MNIST(
    root=DATASETS_ROOT,
    transform=ToTensor(),
)

for i in range(5):
    fold1 = Subset(dataset=mnist_ds, indices=folds[i, 0, :])
    dataset_info(fold1)
    fold2 = Subset(dataset=mnist_ds, indices=folds[i, 1, :])
    dataset_info(fold2)

## FashionMNIST
mean=(0.286,), std=(0.353,)

In [ ]:
from torchvision.datasets import FashionMNIST

fmnist_ds = FashionMNIST(
    root=DATASETS_ROOT,
    transform=ToTensor(),
    download=True,
)
mean, std = get_dataset_mean_std(fmnist_ds)
dataset_info(fmnist_ds)
print(f'{mean=}, {std=}')

In [ ]:
fmnist_ds = undersampling(fmnist_ds, n=5000)

folds = []
for i in range(5):
    f1, f2 = split_data(fmnist_ds, 2)
    folds.append(np.array([fmnist_ds.indices[f1], fmnist_ds.indices[f2]]))
folds = np.array(folds)
np.save(os.path.join(DATASETS_ROOT, 'FashionMNIST', 'folds'), folds)
print(folds.shape)

In [ ]:
folds = np.load(os.path.join(DATASETS_ROOT, 'FashionMNIST', 'folds.npy'))

mnist_ds = FashionMNIST(
    root=DATASETS_ROOT,
    transform=ToTensor(),
)

for i in range(5):
    fold1 = Subset(dataset=mnist_ds, indices=folds[i, 0, :])
    dataset_info(fold1)
    fold2 = Subset(dataset=mnist_ds, indices=folds[i, 1, :])
    dataset_info(fold2)

## CIFAR10
mean=(0.4914, 0.4822, 0.4465), std=(0.247, 0.2435, 0.2616)

In [ ]:
from torchvision.datasets import CIFAR10

cifar10_ds = CIFAR10(
    root=os.path.join(DATASETS_ROOT, 'CIFAR10'),
    transform=ToTensor(),
)
mean, std = get_dataset_mean_std(cifar10_ds)
dataset_info(cifar10_ds)
print(f'{mean=}, {std=}')

In [ ]:
folds = []
for i in range(5):
    folds.append(np.array(split_data(cifar10_ds, 2)))
folds = np.array(folds)
np.save(os.path.join(DATASETS_ROOT, 'CIFAR10', 'folds'), folds)
print(folds.shape)

In [ ]:
folds = np.load(os.path.join(DATASETS_ROOT, 'CIFAR10', 'folds.npy'))

cifar10_ds = CIFAR10(
    root=os.path.join(DATASETS_ROOT, 'CIFAR10'),
    transform=ToTensor(),
)

for i in range(5):
    fold1 = Subset(dataset=cifar10_ds, indices=folds[i, 0, :])
    dataset_info(fold1)
    fold2 = Subset(dataset=cifar10_ds, indices=folds[i, 1, :])
    dataset_info(fold2)

## ImageFolder

In [ ]:
from typing import Optional
from torchvision.datasets import ImageFolder

def check_dataset(dataset: str):
    folds = np.load(os.path.join(DATASETS_ROOT, dataset, 'folds.npy'))
    ds = ImageFolder(
        root=os.path.join(DATASETS_ROOT, dataset),
        transform=ToTensor(),
    )

    for i in range(5):
        fold1 = Subset(dataset=ds, indices=folds[i, 0, :])
        dataset_info(fold1, verbose=True)
        fold2 = Subset(dataset=ds, indices=folds[i, 1, :])
        dataset_info(fold2, verbose=True)

def prepare_dataset(dataset: str, check: bool = True, n: Optional[int] = None):
    ds = ImageFolder(
        root=os.path.join(DATASETS_ROOT, dataset),
        transform=ToTensor(),
    )
    mean, std = get_dataset_mean_std(ds)
    classes = dataset_info(ds)
    print('------------------------------------------------------------------')
    print('dataset info:')
    print('------------------------------------------------------------------')
    idx_to_class = {v: k for k, v in ds.class_to_idx.items()}
    for k, v in classes.items():
        print(f"Class {k} {idx_to_class[k]} contains {v} samples.")
    print(f'{mean=}, {std=}')
    print('------------------------------------------------------------------')
    print('undersamling...')
    print('------------------------------------------------------------------')
    if n is None:
        n = min(classes.values())
        n = n if n % 2 == 0 else n - 1
    ds = undersampling(ds, n=n)
    mean, std = get_dataset_mean_std(ds)
    print(f'{mean=}, {std=}')
    folds = []
    for i in range(5):
        f1, f2 = split_data(ds, 2)
        folds.append(np.array([ds.indices[f1], ds.indices[f2]]))
    folds = np.array(folds)
    np.save(os.path.join(DATASETS_ROOT, dataset, 'folds'), folds)
    print(folds.shape)
    if check:
        print('------------------------------------------------------------------')
        print('checking dataset...')
        print('------------------------------------------------------------------')
        check_dataset(dataset)

## Land-Use_Scene_Classification
mean=(0.462, 0.471, 0.440), std=(0.287, 0.279, 0.269)

In [ ]:
prepare_dataset(os.path.join(DATASETS_ROOT, 'Land-Use_Scene_Classification/images'), n=200)

## Minerals 200x200
mean=(0.54, 0.61, 0.51), std=(0.22, 0.23, 0.23)

In [ ]:
prepare_dataset(os.path.join(DATASETS_ROOT, 'minerals_21_200'), n=200)